In [1]:
#import inbuilt packages
import os
os.chdir("..")
from scipy.stats import bernoulli

from scipy.stats import mvn
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import pdb
import pandas as pd
#from sklearn.preprocessing import LabelEncoder
#from sklearn.utils import shuffle
#from sklearn.model_selection import train_test_split
from scipy.stats import norm
from plots import sortedplot as sp
#from tensorflow.python import debug as tf_debug
#import tensorflow.keras.backend as K
from misc.sample import posterior_sample 
#import importlib as imp
from datagen import GaussianDG 
from datagen import NormalMixDG 
from datagen import DataGenerator
#from data.randomParameters import NormalMixRandomParameters as NMixPar1
from data.randomParameters import NormalMixPNParameters as NMixPar
from random import randint
import seaborn as sns
import pandas as pd

#tf.enable_eager_execution()

In [ ]:
#Vikram, please Start here

#Dataset Generation: Positive set is drawn from N(0,1). The distribution of negatives is a Normal with mean mu,
#standard deviation sig. alpha is the proportion of positives in the Unlabeled set. n_p and n_u are the size of 
#the positive and the unlabeled sets
#pdb.set_trace()
#alpha will be the proportion of positives in the unlabeled set
alpha = 0.05
n_p = 5000
n_u =20000
mu = 2
sig = 1
dg = GaussianDG(mu=mu, sig=sig, alpha=alpha, n_p=n_p, n_u=n_u)
#x contains the one dimensional feature. y contains the PU labels (1: positive, 0:unlabeled) and y_p contains 
#the labels of all positives including the one in the unlabeled set
[x,y,y_p] = dg.pu_data()
#Visualize the PU and PN densities. 

#To generate PN Data
x_p = dg.data_pos(1000)
x_n = dg.data_neg(1000)

#PN posterior. This is what we want to estimate
#sp.sortedplot() sorts the inputs w.r.t the x coordinate before plotting
sp.sortedplot(x, dg.pn_posterior_cc(x))

#PU posterior. This is what we can estimate by training a PU classifier.
sp.sortedplot(x, dg.pu_posterior(x))

sp.sortedplot(x, dg.dens_mix(x, alpha))
sp.sortedplot(x, dg.dens_neg(x))
sp.sortedplot(x, dg.dens_pos(x))
sp.hist(x[(y_p==0).flatten(),:], bins=20, density=True)
sp.hist(x[(y==1).flatten(),:], bins=18, density=True)
sp.show()

In [ ]:
#Dataset Generation: Positive set is drawn from N(0,1). The distribution of negatives is a a mixture of Normals. 
#alpha is the proportion of positives in the Unlabeled set. n_p and n_u are the size of the positive and the 
#unlabeled sets
#pdb.set_trace()
alpha = 0.25
n_pos = 500
n_ul =20000
mu_neg = np.array([-4.0, 4.0])
sig_pos = np.array([1.0])
p_pos = np.array([1.0])
mu_pos = np.array([0.0])
sig_neg = np.array([1.0, 1.0])
p_neg = np.array([0.5, 0.5])

dg = NormalMixDG(mu_pos=mu_pos, sig_pos=sig_pos, p_pos=p_pos, mu_neg=mu_neg, sig_neg=sig_neg, p_neg=p_neg,
                 alpha=alpha, n_pos=n_pos, n_ul=n_ul)
#x contains the one dimensional feature. y contains the PU labels and y_p contains the labels of all positives
#including the one in the unlabeled set
[x,y,y_p] = dg.pu_data()

In [ ]:
#Visualize the PU and PN densities. 

#PN posterior. This is what we want to estimate
#sp.sortedplot() sorts the inputs w.r.t the x coordinate before plotting
sp.sortedplot(x, dg.pn_posterior_cc(x))

#PU posterior. This is what we can estimate by training a PU classifier.
sp.sortedplot(x, dg.pu_posterior(x))

sp.sortedplot(x, dg.dens_mix(x, alpha))
sp.sortedplot(x, dg.dens_neg(x))
sp.sortedplot(x, dg.dens_pos(x))

sp.show()

### Approximating Irreducibility:
We can quantify irreducibility as the probability mass of the region in the support of the positive density, where the negative density is $0$. This region can be defined in terms of the PN posterior as that region, where the PN posterior takes value of $1$. In theory, this is true irrespective of the value of the class prior. We relax the criteria of defining the region with a PN posterior of $1$ and instead define it with PN Posterior achieving value greater than $a$, for some value of $a$ close to 1. 
\begin{align*}
I_a(f_1,f_0) &= \int \mathbf{I}[p(y=1|x)\geq a] f_1(x)dx\\
 &= \mathbf{E}_{f_1}\big[\mathbf{I}[p(y=1|x)\geq a]\big]
\end{align*}
where $p(y=1|x)$ is the balalnced class posterior, i.e.,
\begin{align*}
p(y=1|x) &= \frac{0.5f_1(x)}{0.5f_1(x) + 0.5 f_0(x)} \\
&= \frac{f_1(x)}{f_1(x) + f_0(x)} 
\end{align*}
Notice that $I_1$ is the probability mass of the region in the support of positives the negatives have $0$ density. However, since Gaussian distribution is supported on the entire $R^d$ space, in theory $I_1$ is $0$. By choosing $a$ to be slightly below $1$ we relax the irreducibility criteria. For the data generation below, we used $a=0.95$

In [ ]:
#Create a data generator having positive and negative distribution
#%matplotlib inline
#Set dimensions of the dataset
dim =10
#number of components in the positive and negative distribution
n_comps = 3
#The size of the PN Data for visulization
n=10000
#Create the initial positive negative distributions with components at random locations and identity covariance.
#The distribution of the positives and negatives is identical for the initiallly
NMix = NMixPar(dim, n_comps)
#Compute AUCPN and approximate value of irreducibility.
metrics = NMix.computePNDataMetrics()
print(metrics)
#Generate PN data for visualization. This is the data from initial distributions which has identical 
#positives and negatives
xy, x, y, pos, neg, dg = NMix.generatePNData(n,n)
#Visualize data with pair plots
df = pd.DataFrame(xy)
sns.pairplot(df, vars = np.arange(dim), hue = dim,  plot_kws = {'alpha': 0.1, 's': 20, 'edgecolor': 'k'},
             size = 4)


#Set a desired AUCPN Range. AUCPN is alaways greater then 0.5. I'll suggest starting with values of 0.6, below 
#0.6 the classes are almost identical
aucpn_range = np.array([0.6,0.65])
#Getting parameters in the right AUCPN range and the right Irreducibility range simultaneously is difficult. 
#Keeping the irreducibility range broad makes it easy. I'd suggest keep the irreducibility range to [0.01, 1] 
irreducibility_range = np.array([0.01,1])
#Perturb the parameters of the positive and negative distribution components as well as the the component proportions 
#to achieve desired AUCPN and Irreducibility range
NMix.perturb2SatisfyMetrics(irreducibility_range, aucpn_range)
#Generate PN Data for visulaization. 
xy, x, y, pos, neg, dg = NMix.generatePNData(n,n)
#Compute the metrics
metrics = NMix.computePNDataMetrics()
print(metrics)
#visualize
df = pd.DataFrame(xy)
sns.pairplot(df, vars = np.arange(dim), hue = dim,  plot_kws = {'alpha': 0.1, 's': 20, 'edgecolor': 'k'},
             size = 4)

#Generate the PU Data
n_u = 20000
n_p = 5000
dgPU = NMix.createDataGenerator(n_p, n_u)
#x contains the input features. y contains the PU labels and y_p contains the labels of all positives
#including the one in the unlabeled set
[x,y,y_p] = dgPU.pu_data()
    


In [ ]:
#Code for 3D plot 
ax = plt.axes(projection='3d')
posterior = dg.pn_posterior_balanced(x)
ax.scatter3D(x[:,0], x[:,1], x[:,2], c= posterior > 0.95)
plt.scatter()

In [ ]:
#Multivariate data generator.
from data.randomParameters import NormalMixParameters as NMixPar
def GaussianMixtureDataGenerator(dim, n_comps, aucpn_range):
    irreducibility_range = [0.01, 1]
    NMix = NMixPar(dim, n_comps)
    NMix.perturb2SatisfyMetrics(irreducibility_range, aucpn_range)
    dg = NMix.dg
    return NMix, dg

#Dimension
dim = 10
#Number of components in positves and negatives
n_comps = 10
#The desired range of AUC
aucpn_range = [0.65, 0.7]
#Dataset size
n = 20000
#proportion of positives
alpha = 0.5
NMix, dg = GaussianMixtureDataGenerator(dim, n_comps, aucpn_range)
#x contains the features and y contains the class labels
x, y = dg.pn_data(n, alpha)[0:2]

In [ ]:
[NMix.acceptableSigma(sigma) for sigma in NMix.sig_neg + NMix.sig_pos]

In [ ]:
NMix.sig_neg

In [2]:
#Multivariate data generator.
from data.randomParameters import NormalMixPNParameters2 as NMixPar
def GaussianMixtureDataGenerator(dim, n_comps, aucpn_range):
    irreducibility_range = [0.001, 1]
    NMix = NMixPar(dim, n_comps)
    NMix.perturb2SatisfyMetrics(irreducibility_range, aucpn_range)
    dg = NMix.dg
    return NMix, dg

#Dimension
dim = 4
#Number of components in positves and negatives
n_comps = 4
#The desired range of AUC
aucpn_range = [0.95, 1]
#Dataset size
n = 20000
#proportion of positives
alpha = 0.5
NMix, dg = GaussianMixtureDataGenerator(dim, n_comps, aucpn_range)
#x contains the features and y contains the class labels
x, y = dg.pn_data(n, alpha)[0:2]

{'aucpn': 0.5, 'irreducibility': 0.0}
Sigma Scale Perturb
Sigma Ratios: [0.0625 1.    ]
0.75
Sigma Ratios: [0.31640625 1.        ]
0.875
Sigma Ratios: [0.58618164 1.        ]
Sigma Ratios: [0.58618164 1.        ]
{'aucpn': 0.5475425199999999, 'irreducibility': 0.0}
Sigma Shape Perturb
Sigma Ratios: [1.        0.3798884]
Sigma Ratios: [1.         1.82813111]
Sigma Ratios: [1.         1.82813111]
{'aucpn': 0.7068667200000001, 'irreducibility': 0.0782}
Perturb Proportion
{'aucpn': 0.70857692, 'irreducibility': 0.0826}
Perturb Proportion
{'aucpn': 0.6959369200000001, 'irreducibility': 0.078}
Perturb Proportion
{'aucpn': 0.6931028400000001, 'irreducibility': 0.0724}
Sigma Scale Perturb
Sigma Ratios: [1.         0.11425819]
0.75
Sigma Ratios: [1.         0.57843211]
Sigma Ratios: [1.         0.57843211]
{'aucpn': 0.7153122000000001, 'irreducibility': 0.0282}
Mu Perturb
{'aucpn': 0.8427013999999999, 'irreducibility': 0.0708}
Sigma Scale Perturb
Sigma Ratios: [1.         0.03615201]
0.75
Sigma

{'aucpn': 0.86125244, 'irreducibility': 0.1742}
Sigma Shape Perturb
Sigma Ratios: [1.         0.60900719]
Sigma Ratios: [1.         0.60900719]
{'aucpn': 0.90807792, 'irreducibility': 0.3482}
Sigma Shape Perturb
Sigma Ratios: [0.05657161 1.        ]
Sigma Ratios: [0.97332993 1.        ]
Sigma Ratios: [0.97332993 1.        ]
{'aucpn': 0.91194128, 'irreducibility': 0.3938}
Mu Perturb
{'aucpn': 0.9707435199999999, 'irreducibility': 0.611}


In [3]:
print(dg.dist_pos.mixProp)
print(dg.dist_neg.mixProp)

[0.55573855 0.08193917 0.21405286 0.14826941]
[0.09771288 0.45583994 0.44065592 0.00579125]


In [4]:
a = np.array([[True, False],[False, False]])
np.any(a)

True

In [5]:
from data.randomParameters import AUCFromDistributions
AUCFromDistributions(dg.dist_p, dg.dist_n)

0.9323499199999999

In [ ]:
np.sum()

In [6]:
dg.dist_n.mixProp-dg.dist_p.mixProp
NMix.withinPairAUC()
aucs, cmb = NMix.betweenAllPairsAUC()
aucs

[[0.9942160000000001, 0.9895723200000001, 0.80535024, 0.98008512],
 [0.88880896, 0.99176208, 0.904208, 0.86175296],
 [0.9889296, 0.9448976, 0.8527119999999999, 0.9339496000000002],
 [0.9721222399999999, 0.89886688, 0.99335616, 0.9949361600000001],
 [0.9287542399999998, 0.9642633599999999, 0.99126432, 0.9204299200000001],
 [0.946616, 0.94457232, 0.93084816, 0.9793816]]

In [ ]:
from scipy.stats import dirichlet
dirichlet(0.5*np.ones(1)).rvs([])

In [9]:
from itertools import combinations
#[print(str(i)+str(j)) for i,j in cmb]
cmb = combinations(np.arange(NMix.nComps), 2)
[print(str(i)+str(j)) for i,j in cmb]


01
02
03
12
13
23


[None, None, None, None, None, None]

In [ ]:
from itertools import combinations
for i,j in combinations([1,2,3,4],2):
    print(str(i)+str(j))

In [10]:
[print(str(i)+str(j)) for i,j in cmb]

[]

In [ ]:
NMix.nComps
